In [1]:
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
import pickle

import pandas as pd
import numpy as np

import os
import sys
import io
import shutil

import matplotlib.pyplot as plt

In [2]:
# !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
import cloudscraper
scraper = cloudscraper.create_scraper()

In [3]:
# # !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
# import cfscrape
# scraper = cfscrape.create_scraper()
# # 이후 403 error이 발생한 곳에는 requests 대신 scraper 사용

In [4]:
headers = {'Referer': 'https://kr.investing.com/',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'}

In [5]:
sys.path.append('../../data/constant')
from constants import COMPANY_CODE

In [6]:
def concat_df(df_o, df, dup_col, sort_col):
    df_o.drop_duplicates(subset=[dup_col], keep='last', inplace=True) # dup_col 중첩제거 기준 컬럼 이름: "time", "date" 등
#     df_o.drop_duplicates(subset=[dup_col], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[sort_col]], inplace=True) # sort_col 정렬 기준 컬럼 번호
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

## 일별시세

In [7]:
# 일자별 주식 데이터를 페이지별로 10개씩 취득
def get_piece_date_price(url_d):
    res = scraper.get(url_d, headers=headers)
    class_name = 'type2'
    df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
    
    df = df.dropna(axis=0) # delete nan rows

    df.columns = ['date', 'close', 'close_change', 'open', 'high', 'low', 'volume'] # rename column
    df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y.%m.%d')) # convert character to datetime object
    
    # convert character values to integer value : 보합= 0, 하락= -, 상승= +
    df['close_change'] = df['close_change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                  else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                        else int(x[4:].replace(',',''))))  # convert characters to int
    # define variable types
    df['open'] = df['open'].astype(int)
    df['high'] = df['high'].astype(int)
    df['low'] = df['low'].astype(int)
    df['close'] = df['close'].astype(int)
    df['volume'] = df['volume'].astype(int)
    
    df = df[['date', 'open', 'high', 'low', 'close', 'close_change', 'volume']]  # rearrange columns

    return df

In [8]:
# 10개씩의 일자별 데이터를 원하는 일자부터 현재일자까지 합하어 취득
def get_date_price(url_base_d, code_com):
    
    page_num = 1
    
    # make first data frame
    page = str(page_num)
    url_date = url_base_d + '?code=' + code_com + '&page=' + page
    df_base = get_piece_date_price(url_date)
    
    page_num = page_num + 1

    startdate_str = '2020/1/2 00:00:00' # 데이터 수집 시작 일자, startdate_str
    startdate = datetime.datetime.strptime(startdate_str, '%Y/%m/%d %H:%M:%S')
    
    while True:
        page = str(page_num)
        
        url_date = url_base_d + '?code=' + code_com + '&page=' + page
        df_p = get_piece_date_price(url_date)
        # print("page_num", page_num, end=", ")
        # print("length", len(df_p))
        df_base = concat_df(df_base, df_p, 'date', 0)  # df concat후 'time' column을 기준으로 중복제거 후 0 column을 기준으로 정렬시킴.
        
        if len(df_p) < 10: # 10개 이하가 아니면 완료
            break
        if (startdate == df_base['date']).any(): # 시작일자와 일치하는 row가 있으면 더 이상 진행하지 않음.
            break

        page_num = page_num + 1

    return df_base

In [10]:
# naver_dir = 'data/naver_finance/date_data'
naver_dir = 'date_data'

url_base = 'https://finance.naver.com/item/sise_day.naver'  # sise_day

code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], 
        '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

code_dic = COMPANY_CODE
# code_dic = {'005930': ['삼성전자', 'sec'],}

for i, (code, company_name) in enumerate(code_dic.items()):
    df_collect = get_date_price(url_base, code)
    f_name = f'{naver_dir}/{company_name[1]}.csv'
    df_collect.to_csv(f_name)
    df_collect.to_pickle(f_name.replace('csv','pkl'))
    print(i, end=", ")

0, 1, 2, 

KeyboardInterrupt: 

In [10]:
df_collect.head(10)

,date,open,high,low,close,close_change,volume
0,2024-06-18,78700,80200,78600,79800,1700,18207598
1,2024-06-19,81100,82500,80500,81200,1400,24168863
2,2024-06-20,81500,82200,81200,81600,400,20288913
3,2024-06-21,80700,80800,80000,80000,-1600,17907523
4,2024-06-24,79700,80900,79500,80600,600,15454227
5,2024-06-25,80600,81800,80100,80800,200,19088458
6,2024-06-26,80100,81400,79900,81300,500,17783242
7,2024-06-27,81300,81600,80500,81600,300,11739720
8,2024-06-28,81900,81900,80800,81500,-100,9455929
9,2024-07-01,81500,82100,81300,81800,300,11248358


## 여기까지

In [11]:
url_date = 'https://finance.naver.com/item/sise_day.naver?code=033780&page=117'
df_base = get_piece_date_price(url_date)

startdate_str = '2020/1/2 00:00:00' # 데이터 수집 시작 일자, startdate_str
startdate = datetime.datetime.strptime(startdate_str, '%Y/%m/%d %H:%M:%S')

df_p = get_piece_date_price(url_date)
# print("page_num", page_num, end=", ")
# print("length", len(df_p))

# if len(df_p) < 10: # 10개 이하가 아니면 완료
#     break
# if (startdate == df_base['date']).any(): # 시작일자와 일치하는 row가 있으면 더 이상 진행하지 않음.
#     break



In [12]:
df_p.head()

,date,open,high,low,close,close_change,volume
1,2020-01-02,93800,93900,92400,92500,-1300,448511
2,2019-12-30,94300,94600,93700,93800,-500,339329
3,2019-12-27,94200,94800,94000,94300,-3300,651026
4,2019-12-26,97700,97700,96900,97600,200,639027
5,2019-12-24,97100,97600,97100,97400,200,305867


In [13]:
startdate

datetime.datetime(2020, 1, 2, 0, 0)

In [16]:
df_p['date'].iloc[0]


Timestamp('2020-01-02 00:00:00')

In [27]:
startdate == pd.Timestamp(df_p['date']).any()

TypeError: Cannot convert input [1    2020-01-02
2    2019-12-30
3    2019-12-27
4    2019-12-26
5    2019-12-24
9    2019-12-23
10   2019-12-20
11   2019-12-19
12   2019-12-18
13   2019-12-17
Name: date, dtype: datetime64[ns]] of type <class 'pandas.core.series.Series'> to Timestamp

In [24]:
pd.Timestamp(df_p['date'].any()) # pd.Timestamp(datetime(2014, 1, 23))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22580\1538117694.py:1: FutureWarning: 'any' with datetime64 dtypes is deprecated and will raise in a future version. Use (obj != pd.Timestamp(0)).any() instead.
  pd.Timestamp(df_p['date'].any()) # pd.Timestamp(datetime(2014, 1, 23))


TypeError: Cannot convert input [True] of type <class 'numpy.bool_'> to Timestamp

In [19]:
startdate == df_p['date'].Timestamp(0).any()

AttributeError: 'Series' object has no attribute 'Timestamp'